<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning2/blob/master/Asg2/B_tf-gradient-tape-linear-clfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

B. Low level tensorflow code - Use gradient tape and low level tensorflow code (do not use keras) to build a linear classifier, svm classifier (hint: adding margin to the of linear  loss) and deep neural network classifier. Provide the necessary metrics for performance of classifier as well as visualizations. Also build a deep neural network by subclassing the keras layer class (see slides - not functional api  and not using denselayer class - you need to define your own MyDenseLayer using subclassing    ) and demonstrate the performance of it.   - check code and links in slides for hints

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
def training_step(inputs, targets):
  with tf.GradientTape() as tape:
    predictions = models(inputs)
    loss = square_loss(predictions, targets)
  grad_loss_wrt_W, grad_loss_wrt_b = tape.gradient(loss, [W, b])
  W.assing_sub(grad_loss_wrt_W * learning_rate)
  b.assign_sub(grad_loss_wrt_b * learning_rate)
  return loss

In [ ]:
#https://livebook.manning.com/book/deep-learning-with-python-second-edition/chapter-3/v-3/1